In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 23.7 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM-135M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
data = [
    {"text": "Question: What is AI?\nAnswer: Artificial Intelligence is..."},
    {"text": "Question: What is machine learning?\nAnswer: Machine learning is..."},
]


In [4]:
from datasets import Dataset

dataset = Dataset.from_list(data)


Tokenize Dataset


In [5]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize, remove_columns=["text"])


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Configure LoRA
This allows fine-tuning without full retraining

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414


Training Setup

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./smolllm-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)


Train the Model

In [8]:
tokenized_dataset = tokenized_dataset.map(lambda examples: {"labels": examples["input_ids"]})

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=3, training_loss=14.570157368977865, metrics={'train_runtime': 5.6639, 'train_samples_per_second': 1.059, 'train_steps_per_second': 0.53, 'total_flos': 983017783296.0, 'train_loss': 14.570157368977865, 'epoch': 3.0})

In [9]:
model.save_pretrained("smolllm-lora")
tokenizer.save_pretrained("smolllm-lora")


('smolllm-lora/tokenizer_config.json',
 'smolllm-lora/special_tokens_map.json',
 'smolllm-lora/vocab.json',
 'smolllm-lora/merges.txt',
 'smolllm-lora/added_tokens.json',
 'smolllm-lora/tokenizer.json')

In [11]:
prompt = "Question: Define Politics?\nAnswer:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=50
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: Define Politics?
Answer: Politics is the study of how people organize themselves to make decisions and solve problems. It involves the use of power, persuasion, and communication to achieve political goals.
- Political parties: Political parties are groups of people who come together to support a particular
